# Starter Benchmark Tracker

In [ ]:
from collections import defaultdict
from dataclasses import dataclass, field
from math import log, sqrt

@dataclass
class TrackerBase:
    history: dict[str, list[tuple[int, float]]] = field(default_factory=lambda: defaultdict(list))

    def tick(self, data):
        for asset, points in (data or {}).items():
            self.history.setdefault(asset, []).extend(points or [])

    def predict(self, asset: str, horizon: int, step: int):
        raise NotImplementedError

class GaussianStepTracker(TrackerBase):
    def predict(self, asset: str, horizon: int, step: int):
        prices = [float(price) for _, price in self.history.get(asset, []) if float(price) > 0]
        if len(prices) < 3 or step <= 0:
            return []
        returns = [log(prices[i] / prices[i - 1]) for i in range(1, len(prices)) if prices[i - 1] > 0]
        if len(returns) < 2:
            return []
        mu = sum(returns) / len(returns)
        variance = sum((r - mu) ** 2 for r in returns) / len(returns)
        sigma = sqrt(max(variance, 1e-12))
        segments = max(1, int(horizon // step))
        return [
            {
                'step': k * step,
                'type': 'mixture',
                'components': [
                    {
                        'density': {'type': 'builtin', 'name': 'norm', 'params': {'loc': mu, 'scale': sigma}},
                        'weight': 1,
                    }
                ],
            }
            for k in range(1, segments + 1)
        ]
